<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/NN_Sparse_2_3_Relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.8 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from tabulate import tabulate

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, callbacks, regularizers

from scipy.stats import entropy

# Bayesian Network Data Generation 1000, 2000, ..., 10000 Samples (sparse)

In [6]:
# Function to generate sparse CPDs
def generate_sparse_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random sparse probabilities for EI given IR (introduce zeros or very small values)
    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs[ei_given_ir_probs < 0.5] = 0  # Introduce sparsity; any values less than 0.5 are = 0
    ei_given_ir_probs_sum = ei_given_ir_probs.sum(axis=0, keepdims=True)

    # If any column sums to zero, replace it with uniform probabilities to avoid NaN
    ei_given_ir_probs[:, ei_given_ir_probs_sum[0] == 0] = 1 / 3
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    # Generate random sparse probabilities for SP given IR and EI
    sp_probs = np.random.rand(3, 3, 3)
    sp_probs[sp_probs < 0.2] = 0  # Introduce sparsity
    sp_probs_sum = sp_probs.sum(axis=0, keepdims=True)

    # If any column sums to zero, replace it with uniform probabilities to avoid NaN
    sp_probs[:, sp_probs_sum[0] == 0] = 1 / 3
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, ei_given_ir_probs, sp_probs

# Function to generate and save samples
def generate_and_save_sparse_samples(ir_probs, ei_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample `IR` state based on `IR` probabilities
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample `EI` state based on `EI` probabilities given `IR`
        ei_probs_given_ir = ei_probs[:, ir_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir[ei_state_idx]

        # Sample `SP` state based on `SP` probabilities given `IR` and `EI`
        sp_probs_given_ir_ei = sp_probs[:, ir_state_idx, ei_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei[sp_state_idx]

        # Append sample data to output list
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}'
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated sparse samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes from 50 to 20000
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

for size in sample_sizes:
    # Generate the sparse CPDs
    ir_probs, ei_given_ir_probs, sp_probs = generate_sparse_cpds()

    # Generate and save individual sparse samples for the given sample size
    generate_and_save_sparse_samples(ir_probs, ei_given_ir_probs, sp_probs, size, f'combined_probabilities_sparse_{size}.csv')

# Notify the user that the process is done
print("\nGeneration and saving of individual sparse samples complete for all sample sizes!")


Sample size: 50 - First few rows of generated sparse samples:

+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|    | IR_State   |   IR_Prob | EI_State   |   EI_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability |
+====+============+===========+============+===========+=================================================+===================+=========================+
|  0 | low        |    0.322  | poor       |    0.5246 | 0.3718, 0.1673, 0.4609                          | stable            |                  0.1673 |
+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|  1 | high       |    0.4229 | poor       |    1      | 0.4988, 0.3101, 0.1911                          | stable            |                  0.3101 |
+----+------------

# Hypothesis Model 500, ..., 20000 Samples (sparse) 1 hidden Layer, 3 Neurons Relu & K-L

In [7]:
# Sample sizes to loop through
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

# Prepare a list to store K-L divergence results
kl_divergence_results = []

# Define the Neural Network architecture with L2 regularization
def create_nn_model(hidden_layers=2, nodes_per_layer=3, l2_lambda=0.01):
    model = models.Sequential()

    # Input layer (2 input features: IR_encoded and EI_encoded)
    model.add(layers.InputLayer(input_shape=(2,)))

    # Hidden layers with L2 regularization and Dropout
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(
            nodes_per_layer,
            activation='relu',
            kernel_regularizer=regularizers.l2(l2_lambda),  # L2 regularization
            name=f"hidden_layer_{layer_num + 1}"
        ))
        model.add(layers.Dropout(0.2))  # Dropout layer to reduce overfitting

    # Output layer (3 classes: decrease, stable, increase) with L2 regularization
    model.add(layers.Dense(
        3,
        activation='softmax',
        kernel_regularizer=regularizers.l2(l2_lambda),  # L2 regularization
        name="output_layer"
    ))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Loop through each sample size
for size in sample_sizes:
    print(f"\nSample size: {size}")

    # Load the sparse dataset for the current sample size
    combined_data_bn = pd.read_csv(f'combined_probabilities_sparse_{size}.csv')

    # Encode categorical variables
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    combined_data_bn['IR_encoded'] = combined_data_bn['IR_State'].map(ir_map)
    combined_data_bn['EI_encoded'] = combined_data_bn['EI_State'].map(ei_map)
    combined_data_bn['SP_encoded'] = combined_data_bn['Chosen_SP_State'].map(sp_map)

    # Split the data into features (X) and labels (y)
    X = combined_data_bn[['IR_encoded', 'EI_encoded']]
    y = combined_data_bn['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model
    nn_model = create_nn_model(hidden_layers=2, nodes_per_layer=3, l2_lambda=0.01)

    # Early stopping callback to prevent overfitting
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=25,
                           batch_size=16,
                           validation_data=(X_val, y_val),
                           callbacks=[early_stopping],
                           verbose=0)

    # Print training, validation, and test accuracy
    train_loss, train_accuracy = nn_model.evaluate(X_train, y_train, verbose=0)
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Training Accuracy for {size} samples: {train_accuracy:.4f}")
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the IR, EI, predicted SP, and the NN probabilities
    result_df = pd.DataFrame({
        'IR': combined_data_bn['IR_State'].iloc[X_test.index],  # IR column from the original dataframe for the test set
        'EI': combined_data_bn['EI_State'].iloc[X_test.index],  # EI column from the original dataframe for the test set
        'Predicted_SP': predicted_labels           # Predicted SP labels
    })

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df.reset_index(drop=True), probs_df.reset_index(drop=True)], axis=1)

    # Save the test data with predictions to a CSV file
    combined_df.to_csv(f'test_data_nn_sparse_{size}.csv', index=False)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 15 rows):")
    print(combined_df.head(15))

    # --- K-L Divergence and Standard Deviation Calculation Block ---
    # Extract NN predicted probabilities and BN ground truth probabilities
    nn_probs = combined_df[['Prob_decrease', 'Prob_stable', 'Prob_increase']].values
    bn_probs = combined_data_bn.loc[X_test.index, 'SP_Probabilities (decrease, stable, increase)'].apply(
        lambda x: np.array(list(map(float, x.strip('[]').split(','))))
    ).values

    # Calculate K-L divergence between NN predicted probabilities and BN ground truth probabilities
    kl_divergences = []
    for i in range(len(nn_probs)):
        nn_prob = nn_probs[i]
        bn_prob = bn_probs[i]

        # Ensure both are valid probability distributions
        epsilon = 1e-10
        nn_prob = np.clip(nn_prob, epsilon, 1)
        bn_prob = np.clip(bn_prob, epsilon, 1)

        # Normalize to ensure they sum to 1
        nn_prob /= nn_prob.sum()
        bn_prob /= bn_prob.sum()

        # Compute K-L divergence
        kl_div = entropy(bn_prob, nn_prob)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence and standard deviation for this sample size
    average_kl_divergence = np.mean(kl_divergences)
    std_kl_divergence = np.std(kl_divergences)
    print(f"\nK-L Divergence for {size} samples: {average_kl_divergence:.4f}, Std Dev: {std_kl_divergence:.4f}")

    # Append the results to the list
    kl_divergence_results.append({
        'Sample_Size': size,
        'Average_KL_Divergence': average_kl_divergence,
        'Std_Dev': std_kl_divergence
    })

# Save the K-L divergence results to a CSV file
kl_divergence_df = pd.DataFrame(kl_divergence_results)
kl_divergence_df.to_csv('kl_div_NN_sparse.csv', index=False)

print("\nAll sample sizes have been processed and K-L divergences calculated. Results saved to 'kl_div_nn_sparse.csv'.")


Sample size: 50
Training Data: (35, 2) (35,)
Validation Data: (7, 2) (7,)
Test Data: (8, 2) (8,)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 50 samples: 0.3714
Validation Accuracy for 50 samples: 0.5714
Test Accuracy for 50 samples: 0.5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step

Predicted Results and Probabilities for 50 samples (First 15 rows):
       IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0    high     poor     decrease       0.337493     0.336329       0.326178
1    high     poor     decrease       0.337493     0.336329       0.326178
2  medium  average     decrease       0.420404     0.306634       0.272962
3    high     poor     decrease       0.337493     0.336329       0.326178
4  medium  average     decrease       0.420404     0.306634       0.272962
5    high     poor     decrease       0.337493     0.336329       0.326178
6    high     poor     decrease       0.337493     0.336329       0.326178
7    high     poor     decrease       0.337493     0.336329       0.326178

K-L Divergence for 50 samples: 0.0636, Std Dev: 0.0068

Sample size: 100
Training Data: (70, 2) 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 100 samples: 0.4857
Validation Accuracy for 100 samples: 0.4667
Test Accuracy for 100 samples: 0.4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step

Predicted Results and Probabilities for 100 samples (First 15 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low     poor       stable       0.305174     0.354241       0.340585
1   medium  average       stable       0.261783     0.460542       0.277675
2   medium     poor       stable       0.313771     0.381506       0.304723
3   medium     poor       stable       0.313771     0.381506       0.304723
4      low  average       stable       0.273802     0.391988       0.334210
5      low     poor       stable       0.305174     0.354241       0.340585
6   medium     good       stable       0.212587     0.541131       0.246282
7     high     good       stable       0.232671     0.554242       0.213087
8      low  average       stable       0.273802     0.391988       0.334210
9      l

...

Training Accuracy for 18000 samples: 0.4364
Validation Accuracy for 18000 samples: 0.4419
Test Accuracy for 18000 samples: 0.4248
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Predicted Results and Probabilities for 18000 samples (First 15 rows):
        IR    EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low  good     decrease       0.435581      0.38649        0.17793
1   medium  poor     decrease       0.435581      0.38649        0.17793
2      low  good     decrease       0.435581      0.38649        0.17793
3   medium  poor     decrease       0.435581      0.38649        0.17793
4      low  good     decrease       0.435581      0.38649        0.17793
5      low  good     decrease       0.435581      0.38649        0.17793
6     high  poor     decrease       0.435581      0.38649        0.17793
7   medium  poor     decrease       0.435581      0.38649        0.17793
8     high  poor     decrease       0.435581      0.38649        0.17793
9      low  good     decrease

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 19000 samples: 0.4920
Validation Accuracy for 19000 samples: 0.4825
Test Accuracy for 19000 samples: 0.4947
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 19000 samples (First 15 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high     good     increase       0.283286     0.228233       0.488481
1     high     good     increase       0.283286     0.228233       0.488481
2     high     good     increase       0.283286     0.228233       0.488481
3   medium     good     increase       0.283286     0.228233       0.488481
4     high     good     increase       0.283286     0.228233       0.488481
5   medium  average     increase       0.283286     0.228233       0.488481
6     high     good     increase       0.283286     0.228233       0.488481
7   medium     good     increase       0.283286     0.228233       0.488481
8     high     good     increase       0.283286     0.228233       0.488481

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 20000 samples: 0.5227
Validation Accuracy for 20000 samples: 0.5087
Test Accuracy for 20000 samples: 0.5050
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 20000 samples (First 15 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low     poor     decrease       0.465013     0.141965       0.393023
1     high  average     increase       0.184745     0.393990       0.421265
2      low     poor     decrease       0.465013     0.141965       0.393023
3   medium     good     increase       0.317411     0.248610       0.433978
4      low     poor     decrease       0.465013     0.141965       0.393023
5   medium     good     increase       0.317411     0.248610       0.433978
6     high     good     increase       0.276780     0.286718       0.436502
7   medium  average     increase       0.240589     0.324944       0.434467
8      low     poor     decrease       0.465013     0.141965       0.393023